In [1]:
%pip install ydata_profiling
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from math import ceil
from ydata_profiling import ProfileReport

Note: you may need to restart the kernel to use updated packages.


In [2]:
train_data = pd.read_csv('/Users/daan/Desktop/Nova IMS/Machine Learning/Project/project_data/train_data.csv', sep= ",", index_col='Claim Identifier')
pd.set_option('display.max_columns', None)
test_data = pd.read_csv('/Users/daan/Documents/GitHub/ML_Group36/src/test_data.csv', index_col='Claim Identifier')
train_data.head(5)

/var/folders/cb/8gnf1rrd7914vbmqtgrt2mmm0000gn/T/ipykernel_88534/204275892.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('/Users/daan/Desktop/Nova IMS/Machine Learning/Project/project_data/train_data.csv', sep= ",", index_col='Claim Identifier')


,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,NaN,M,NaN,44.0,RETAIL TRADE,I,NaN,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,2020-02-21,F,4.0,23.0,CONSTRUCTION,I,NaN,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,NaN,M,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
957648180,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5393887,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,2A. SIF,2. NON-COMP,DUTCHESS,N,ALBANY,NaN,M,NaN,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,NaN,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [3]:
#drop target variable
knn_df = train_data.drop(columns='Claim Injury Type')

In [4]:
knn_df.drop('OIICS Nature of Injury Description', axis=1,inplace=True)

In [ ]:
#duplicated values
knn_df.duplicated().sum()

In [15]:
knn_method = train_data

In [16]:
#duplicated values
train_data.duplicated().sum()

18350

In [17]:
only_assembly_date = knn_method.drop(columns=['Assembly Date']).isnull().all(axis=1) & knn_method['Assembly Date'].notnull()


num_only_assembly_date_filled = only_assembly_date.sum()
print(f"Number of rows with only 'Assembly Date' filled: {num_only_assembly_date_filled}")

Number of rows with only 'Assembly Date' filled: 19445


In [18]:
knn_method.isnull().sum()/knn_method.shape[0]*100

Accident Date                           3.898084
Age at Injury                           3.276487
Alternative Dispute Resolution          3.276487
Assembly Date                           0.000000
Attorney/Representative                 3.276487
Average Weekly Wage                     8.104187
Birth Year                              8.176137
C-2 Date                                5.729850
C-3 Date                               68.449174
Carrier Name                            3.276487
Carrier Type                            3.276487
Claim Injury Type                       3.276487
County of Injury                        3.276487
COVID-19 Indicator                      3.276487
District Name                           3.276487
First Hearing Date                     74.590502
Gender                                  3.276487
IME-4 Count                            77.622664
Industry Code                           4.954412
Industry Code Description               4.954412
Medical Fee Region  

In [19]:
knn_method.describe(include='O')

,Accident Date,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Description,WCIO Nature of Injury Description,WCIO Part Of Body Description,Zip Code,WCB Decision
count,570337,574026,593471,574026,559466,187245,574026,574026,574026,574026,574026,574026,150798,574026,564068,574026,558386,558369,556944,545389,574026
unique,5539,3,1096,2,2475,1648,2046,8,8,63,2,8,1094,4,20,5,74,56,54,10060,1
top,2020-03-01,N,2020-03-06,N,2021-05-11,2021-04-21,STATE INSURANCE FUND,1A. PRIVATE,2. NON-COMP,SUFFOLK,N,NYC,2022-04-15,M,HEALTH CARE AND SOCIAL ASSISTANCE,IV,LIFTING,STRAIN OR TEAR,LOWER BACK AREA,11236,Not Work Related
freq,1245,571412,1422,392291,1847,350,111144,285368,291078,60430,546505,270779,440,335218,114339,265981,46610,153373,51862,3302,574026


In [20]:
knn_method = knn_method.drop(columns=["OIICS Nature of Injury Description"])
knn_method = knn_method.drop(columns=["Average Weekly Wage"])
knn_method = knn_method.drop(columns=["IME-4 Count"])
knn_method = knn_method.drop(columns=["First Hearing Date"])
knn_method = knn_method.drop(columns=["C-3 Date"])
knn_method.head(5)

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Birth Year,C-2 Date,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,Gender,Industry Code,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,2019-12-30,31.0,N,2020-01-01,N,1988.0,2019-12-31,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,2. NON-COMP,ST. LAWRENCE,N,SYRACUSE,M,44.0,RETAIL TRADE,I,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
5393091,2019-08-30,46.0,N,2020-01-01,Y,1973.0,2020-01-01,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,4. TEMPORARY,WYOMING,N,ROCHESTER,F,23.0,CONSTRUCTION,I,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
5393889,2019-12-06,40.0,N,2020-01-01,N,1979.0,2020-01-01,INDEMNITY INSURANCE CO OF,1A. PRIVATE,4. TEMPORARY,ORANGE,N,ALBANY,M,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
957648180,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5393887,2019-12-30,61.0,N,2020-01-01,N,1958.0,2019-12-31,STATE INSURANCE FUND,2A. SIF,2. NON-COMP,DUTCHESS,N,ALBANY,M,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [21]:
knn_method.isnull().sum()/knn_method.shape[0]*100

Accident Date                        3.898084
Age at Injury                        3.276487
Alternative Dispute Resolution       3.276487
Assembly Date                        0.000000
Attorney/Representative              3.276487
Birth Year                           8.176137
C-2 Date                             5.729850
Carrier Name                         3.276487
Carrier Type                         3.276487
Claim Injury Type                    3.276487
County of Injury                     3.276487
COVID-19 Indicator                   3.276487
District Name                        3.276487
Gender                               3.276487
Industry Code                        4.954412
Industry Code Description            4.954412
Medical Fee Region                   3.276487
WCIO Cause of Injury Code            5.911831
WCIO Cause of Injury Description     5.911831
WCIO Nature of Injury Code           5.914695
WCIO Nature of Injury Description    5.914695
WCIO Part Of Body Code            

In [22]:
knn_method.info()

<class 'pandas.core.frame.DataFrame'>
Index: 593471 entries, 5393875 to 818961390
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Accident Date                      570337 non-null  object 
 1   Age at Injury                      574026 non-null  float64
 2   Alternative Dispute Resolution     574026 non-null  object 
 3   Assembly Date                      593471 non-null  object 
 4   Attorney/Representative            574026 non-null  object 
 5   Birth Year                         544948 non-null  float64
 6   C-2 Date                           559466 non-null  object 
 7   Carrier Name                       574026 non-null  object 
 8   Carrier Type                       574026 non-null  object 
 9   Claim Injury Type                  574026 non-null  object 
 10  County of Injury                   574026 non-null  object 
 11  COVID-19 Indicator                 

In [23]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset (assuming it’s already loaded into `knn_method`)
# knn_method = pd.read_csv("your_dataset.csv")  # Uncomment if loading from CSV

# Step 1: Identify Columns with Missing Values
missing_columns = knn_method.columns[knn_method.isnull().any()]
print("Columns with missing values:", missing_columns)

Columns with missing values: Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Attorney/Representative', 'Birth Year', 'C-2 Date', 'Carrier Name',
       'Carrier Type', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'Gender', 'Industry Code',
       'Industry Code Description', 'Medical Fee Region',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')


In [24]:
# Step 2: Encode Categorical Data
label_encoders = {}
for column in knn_method.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    knn_method[column] = knn_method[column].astype(str)  # Ensure all values are strings
    knn_method[column] = le.fit_transform(knn_method[column])
    label_encoders[column] = le  # Save encoder for potential reverse transform


In [25]:
# Step 3: Normalize Numerical Features
scaler = StandardScaler()
numerical_columns = knn_method.select_dtypes(include=['float64', 'int64']).columns
knn_method[numerical_columns] = scaler.fit_transform(knn_method[numerical_columns])


In [27]:
# Step 4: Apply KNN Imputation with 'nan_euclidean' metric
imputer = KNNImputer(n_neighbors=5, weights="uniform", metric="nan_euclidean")
knn_imputed = imputer.fit_transform(knn_method)




In [28]:
# Convert the numpy array back to DataFrame
knn_imputed_df = pd.DataFrame(knn_imputed, columns=knn_method.columns)

In [29]:
# Reverse scaling and label encoding if necessary
knn_imputed_df[numerical_columns] = scaler.inverse_transform(knn_imputed_df[numerical_columns])
for column, le in label_encoders.items():
    knn_imputed_df[column] = le.inverse_transform(knn_imputed_df[column].astype(int))

print("Imputation completed.")

Imputation completed.


In [32]:
knn_imputed_df.isnull().sum()

Accident Date                        0
Age at Injury                        0
Alternative Dispute Resolution       0
Assembly Date                        0
Attorney/Representative              0
Birth Year                           0
C-2 Date                             0
Carrier Name                         0
Carrier Type                         0
Claim Injury Type                    0
County of Injury                     0
COVID-19 Indicator                   0
District Name                        0
Gender                               0
Industry Code                        0
Industry Code Description            0
Medical Fee Region                   0
WCIO Cause of Injury Code            0
WCIO Cause of Injury Description     0
WCIO Nature of Injury Code           0
WCIO Nature of Injury Description    0
WCIO Part Of Body Code               0
WCIO Part Of Body Description        0
Zip Code                             0
Agreement Reached                    0
WCB Decision             